In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
pd.options.display.max_columns = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

#Data Wrangling: Join, Combine, and Reshape

 # Hierarchical Indexing
 Hierarchical indexing is an important feature of pandas that enables you to have
 multiple (two or more) index levels on an axis. Another way of thinking about it
 is that it provides a way for you to work with higher dimensional data in a lower
 dimensional form. Let’s start with a simple example: create a Series with a list of lists
 (or arrays) as the index:

In [ ]:
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

 What you’re seeing is a prettified view of a Series with a MultiIndex as its index. The
 “gaps” in the index display mean “use the label directly above”:

In [ ]:
data.index

With a hierarchically indexed object, so-called partial indexing is possible, enabling
 you to concisely select subsets of the data:

In [ ]:
data["b"]
data["b":"c"]
data.loc[["b", "d"]]

 Selection is even possible from an “inner” level. Here I select all of the values having
 the value 2 from the second index level:

In [ ]:
data.loc[:, 2]

 Hierarchical indexing plays an important role in reshaping data and in group-based
 operations like forming a pivot table. For example, you can rearrange this data into a
 DataFrame using its unstack method:

In [ ]:
data.unstack()

 The inverse operation of unstack is stack:

In [ ]:
data.unstack().stack()

With a DataFrame, either axis can have a hierarchical index:

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])
frame

 The hierarchical levels can have names (as strings or any Python objects). If so, these
 will show up in the console output

In [ ]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

 You can see how many levels an index has by accessing its nlevels attribute

In [ ]:
frame.index.nlevels

 With partial column indexing you can similarly select groups of columns

In [ ]:
frame["Ohio"]

# Reordering and Sorting Levels
At times you may need to rearrange the order of the levels on an axis or sort the data
 by the values in one specific level. The swaplevel method takes two level numbers
 or names and returns a new object with the levels interchanged (but the data is
 otherwise unaltered):

In [ ]:
frame.swaplevel("key1", "key2")

 sort_index by default sorts the data lexicographically using all the index levels, but
 you can choose to use only a single level or a subset of levels to sort by passing the
 level argument. For example:

In [ ]:
frame.sort_index(level=1)
frame.swaplevel(0, 1).sort_index(level=0)

 # Summary Statistics by Level

 Many descriptive and summary statistics on DataFrame and Series have a level
 option in which you can specify the level you want to aggregate by on a particular
 axis. Consider the above DataFrame; we can aggregate by level on either the rows or
 columns, like so:



In [ ]:
frame.groupby(level="key2").sum()
frame.groupby(level="color", axis="columns").sum()

 # Indexing with a DataFrame’s columns

  It’s not unusual to want to use one or more columns from a DataFrame as the
 row index; alternatively, you may wish to move the row index into the DataFrame’s
 columns. Here’s an example DataFrame:

In [ ]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame

 DataFrame’s set_index function will create a new DataFrame using one or more of
 its columns as the index:

In [ ]:
frame2 = frame.set_index(["c", "d"])
frame2

By default, the columns are removed from the DataFrame, though you can leave
 them in by passing drop=False to set_index:

In [ ]:
frame.set_index(["c", "d"], drop=False)

 reset_index, on the other hand, does the opposite of set_index; the hierarchical
 index levels are moved into the columns:

In [ ]:
frame2.reset_index()

# Combining and Merging Datasets
 Data contained in pandas objects can be combined in a number of ways:
 # pandas.merge
 Connect rows in DataFrames based on one or more keys. This will be familiar
 to users of SQL or other relational databases, as it implements database join
 operations.
 # pandas.concat
 Concatenate or “stack” objects together along an axis.
 # combine_first
 Splice together overlapping data to fill in missing values in one object with values
 from another.
 I will address each of these and give a number of examples. They’ll be utilized in
 examples throughout the rest of the book.

# Database-Style DataFrame Joins
 Merge or join operations combine datasets by linking rows using one or more keys.
 These operations are particularly important in relational databases (e.g., SQL-based).
 The pandas.merge function in pandas is the main entry point for using these algo
rithms on your data.
 Let’s start with a simple example:

In [ ]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
df1
df2

 This is an example of a many-to-one join; the data in df1 has multiple rows labeled
 a and b, whereas df2 has only one row for each value in the key column. Calling
 pandas.merge with these objects, we obtain:

In [ ]:
pd.merge(df1, df2)

Note that I didn’t specify which column to join on. If that information is not
 specified, pandas.merge uses the overlapping column names as the keys. It’s a good
 practice to specify explicitly, though:

In [ ]:
pd.merge(df1, df2, on="key")

 In general, the order of column output in pandas.merge operations is unspecified.
 If the column names are different in each object, you can specify them separately:

In [ ]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

 You may notice that the "c" and "d" values and associated data are missing from
 the result. By default, pandas.merge does an "inner" join; the keys in the result are
 the intersection, or the common set found in both tables. Other possible options are
 "left", "right", and "outer". The outer join takes the union of the keys, combining
 the effect of applying both left and right joins

In [ ]:
pd.merge(df1, df2, how="outer")
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

 how="inner" Use only the key combinations observed in both tables

 how="left" Use all key combinations found in the left table

 how="right" Use all key combinations found in the right table

 how="outer" Use all key combinations observed in both tables together


In [ ]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": pd.Series(range(6), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": pd.Series(range(5), dtype="Int64")})
df1
df2
pd.merge(df1, df2, on="key", how="left")

 Since there were three "b" rows in the left DataFrame and two in the right one, there
 are six "b" rows in the result. The join method passed to the how keyword argument
 affects only the distinct key values appearing in the result:

In [ ]:
pd.merge(df1, df2, how="inner")

 To merge with multiple keys, pass a list of column names:

In [ ]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})
pd.merge(left, right, on=["key1", "key2"], how="outer")

A last issue to consider in merge operations is the treatment of overlapping column
 names. For example:

In [ ]:
pd.merge(left, right, on="key1")

 While you can address the overlap manually (see the section “Renaming Axis
 Indexes” on page 214 for renaming axis labels), pandas.merge has a suffixes option
 for specifying strings to append to overlapping names in the left and right DataFrame
 objects:

In [ ]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

 # Merging on Index
 In some cases, the merge key(s) in a DataFrame will be found in its index (row
 labels). In this case, you can pass left_index=True or right_index=True (or both) to
 indicate that the index should be used as the merge key:

In [ ]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
left1
right1
pd.merge(left1, right1, left_on="key", right_index=True)

 Since the default merge method is to intersect the join keys, you can instead form the
 union of them with an outer join:

In [ ]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

 With hierarchically indexed data, things are more complicated, as joining on index is
 equivalent to a multiple-key merge:

In [ ]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                               "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})
righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                           index=righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                           index=righth_index)})
lefth
righth

 In this case, you have to indicate multiple columns to merge on as a list (note the
 handling of duplicate index values with how="outer"):

In [ ]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)
pd.merge(lefth, righth, left_on=["key1", "key2"],
         right_index=True, how="outer")

 Using the indexes of both sides of the merge is also possible:

In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"]).astype("Int64")
left2
right2
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

 DataFrame has a join instance method to simplify merging by index. It can also be
 used to combine many DataFrame objects having the same or similar indexes but
 nonoverlapping columns. In the prior example, we could have written:

In [ ]:
left2.join(right2, how="outer")

Compared with pandas.merge, DataFrame’s join method performs a left join on the
 join keys by default. It also supports joining the index of the passed DataFrame on
 one of the columns of the calling DataFrame:

In [ ]:
left1.join(right1, on="key")

 You can think of this method as joining data “into” the object whose join method
 was called.
 Lastly, for simple index-on-index merges, you can pass a list of DataFrames to join
 as an alternative to using the more general pandas.concat function described in the
 next section:

In [ ]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=["a", "c", "e", "f"],
                       columns=["New York", "Oregon"])
another
left2.join([right2, another])
left2.join([right2, another], how="outer")

 # Concatenating Along an Axis

 Another kind of data combination operation is referred to interchangeably as concat
enation or stacking. NumPy’s concatenate function can do this with NumPy arrays:

In [ ]:
arr = np.arange(12).reshape((3, 4))
arr
np.concatenate([arr, arr], axis=1)

In the context of pandas objects such as Series and DataFrame, having labeled axes
 enable you to further generalize array concatenation. In particular, you have a num
ber of additional concerns:


 • If the objects are indexed differently on the other axes, should we combine the
 distinct elements in these axes or use only the values in common?


 • Do the concatenated chunks of data need to be identifiable as such in the result
ing object?


 • Does the “concatenation axis” contain data that needs to be preserved? In
 many cases, the default integer labels in a DataFrame are best discarded during
 concatenation.

 The concat function in pandas provides a consistent way to address each of these
 questions. I’ll give a number of examples to illustrate how it works. Suppose we have
 three Series with no index overlap:

In [ ]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

 Calling pandas.concat with these objects in a list glues together the values and
 indexes:

In [ ]:
s1
s2
s3
pd.concat([s1, s2, s3])

 By default, pandas.concat works along axis="index", producing another Series. If
 you pass axis="columns", the result will instead be a DataFrame:

In [ ]:
pd.concat([s1, s2, s3], axis="columns")

 In this case there is no overlap on the other axis, which as you can see is the
 union (the "outer" join) of the indexes. You can instead intersect them by passing
 join="inner":

In [ ]:
s4 = pd.concat([s1, s3])
s4
pd.concat([s1, s4], axis="columns")
pd.concat([s1, s4], axis="columns", join="inner")

 In this last example, the "f" and "g" labels disappeared because of the join="inner"
 option.

A potential issue is that the concatenated pieces are not identifiable in the result.
 Suppose instead you wanted to create a hierarchical index on the concatenation axis.
 To do this, use the keys argument:

In [ ]:
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])
result
result.unstack()

 In the case of combining Series along axis="columns", the keys become the Data
Frame column headers:

In [ ]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

 The same logic extends to DataFrame objects:

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
                   columns=["one", "two"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
                   columns=["three", "four"])
df1
df2
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

 Here the keys argument is used to create a hierarchical index where the first level can
 be used to identify each of the concatenated DataFrame objects.
 If you pass a dictionary of objects instead of a list, the dictionary’s keys will be used
 for the keys option:

In [ ]:
pd.concat({"level1": df1, "level2": df2}, axis="columns")

 There are additional arguments governing how the hierarchical index is created
 (see Table 8-3). For example, we can name the created axis levels with the names
 argument:

In [ ]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"],
          names=["upper", "lower"])

 A last consideration concerns DataFrames in which the row index does not contain
 any relevant data:

In [ ]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                   columns=["a", "b", "c", "d"])
df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                   columns=["b", "d", "a"])
df1
df2

 In this case, you can pass ignore_index=True, which discards the indexes from each
 DataFrame and concatenates the data in the columns only, assigning a new default
 index:

In [ ]:
pd.concat([df1, df2], ignore_index=True)

 # Combining Data with Overlap
 There is another data combination situation that can’t be expressed as either a merge
 or concatenation operation. You may have two datasets with indexes that overlap in
 full or in part. As a motivating example, consider NumPy’s where function, which
 performs the array-oriented equivalent of an if-else expression:

In [ ]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=["a", "b", "c", "d", "e", "f"])
a
b
np.where(pd.isna(a), b, a)

 Here, whenever values in a are null, values from b are selected, otherwise the non
null values from a are selected. Using numpy.where does not check whether the index
 labels are aligned or not (and does not even require the objects to be the same
 length), so if you want to line up values by index, use the Series combine_first
 method:

In [ ]:
a.combine_first(b)

 # Reshaping and Pivoting
 There are a number of basic operations for rearranging tabular data. These are
 referred to as reshape or pivot operations.

 # Reshaping with Hierarchical Indexing
 Hierarchical indexing provides a consistent way to rearrange data in a DataFrame.
 There are two primary actions:
 # stack
 This “rotates” or pivots from the columns in the data to the rows.
 # unstack
 This pivots from the rows into the columns.
 With DataFrames, combine_first does the same thing column by column, so you
 can think of it as “patching” missing data in the calling object with data from the
 object you pass:

In [ ]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})
df1
df2
df1.combine_first(df2)

I’ll illustrate these operations through a series of examples. Consider a small Data
Frame with string arrays as row and column indexes:

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

 Using the stack method on this data pivots the columns into the rows, producing a
 Series:

In [ ]:
result = data.stack()
result

 From a hierarchically indexed Series, you can rearrange the data back into a Data
Frame with unstack:

In [ ]:
result.unstack()

 By default, the innermost level is unstacked (same with stack). You can unstack a
 different level by passing a level number or name:

In [ ]:
result.unstack(level=0)
result.unstack(level="state")

 Unstacking might introduce missing data if all of the values in the level aren’t found
 in each subgroup:

In [ ]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

 Stacking filters out missing data by default, so the operation is more easily invertible:

In [ ]:
data2.unstack()
data2.unstack().stack()
data2.unstack().stack(dropna=False)

 When you unstack in a DataFrame, the level unstacked becomes the lowest level in
 the result

In [ ]:
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))
df
df.unstack(level="state")

 As with unstack, when calling stack we can indicate the name of the axis to stack:

In [ ]:
df.unstack(level="state").stack(level="side")

# Pivoting “Long” to “Wide” Format
 A common way to store multiple time series in databases and CSV files is what
 is sometimes called long or stacked format. In this format, individual values are
 represented by a single row in a table rather than multiple values per row.

Let’s load some example data and do a small amount of time series wrangling and
 other data cleaning:

In [ ]:
data = pd.read_csv("examples/macrodata.csv")
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
data.head()

 First, I use pandas.PeriodIndex (which represents time intervals rather than points
 in time), discussed in more detail in Chapter 11, to combine the year and quarter
 columns to set the index to consist of datetime values at the end of each quarter:

In [ ]:
periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date")
periods
data.index = periods.to_timestamp("D")
data.head()

 Here I used the pop method on the DataFrame, which returns a column while
 deleting it from the DataFrame at the same time.
 Then, I select a subset of columns and give the columns index the name "item":

In [ ]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

 Lastly, I reshape with stack, turn the new index levels into columns with
 reset_index, and finally give the column containing the data values the name
 "value":

In [ ]:
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))

 Now, ldata looks like:

In [ ]:
long_data[:10]

 In this so-called long format for multiple time series, each row in the table represents
 a single observation.

 Data is frequently stored this way in relational SQL databases, as a fixed schema (col
umn names and data types) allows the number of distinct values in the item column
 to change as data is added to the table. In the previous example, date and item would
 usually be the primary keys (in relational database parlance), offering both relational
 integrity and easier joins. In some cases, the data may be more difficult to work with
 in this format; you might prefer to have a DataFrame containing one column per
 distinct item value indexed by timestamps in the date column. DataFrame’s pivot
 method performs exactly this transformation:

In [ ]:
pivoted = long_data.pivot(index="date", columns="item",
                          values="value")
pivoted.head()

In [ ]:
long_data.index.name = None

 The first two values passed are the columns to be used, respectively, as the row and
 column index, then finally an optional value column to fill the DataFrame. Suppose
 you had two value columns that you wanted to reshape simultaneously:

In [ ]:
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data[:10]

 By omitting the last argument, you obtain a DataFrame with hierarchical columns

In [ ]:
pivoted = long_data.pivot(index="date", columns="item")
pivoted.head()
pivoted["value"].head()

Note that pivot is equivalent to creating a hierarchical index using set_index fol
lowed by a call to unstack:

In [ ]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

 # Pivoting “Wide” to “Long” Format
 An inverse operation to pivot for DataFrames is pandas.melt. Rather than trans
forming one column into many in a new DataFrame, it merges multiple columns into
 one, producing a DataFrame that is longer than the input. Let’s look at an example:

In [ ]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})
df

 The "key" column may be a group indicator, and the other columns are data values.
 When using pandas.melt, we must indicate which columns (if any) are group indica
tors. Let’s use "key" as the only group indicator here:

In [ ]:
melted = pd.melt(df, id_vars="key")
melted

Using pivot, we can reshape back to the original layout

In [ ]:
reshaped = melted.pivot(index="key", columns="variable",
                        values="value")
reshaped

 Since the result of pivot creates an index from the column used as the row labels, we
 may want to use reset_index to move the data back into a column:

In [ ]:
reshaped.reset_index()

 You can also specify a subset of columns to use as value columns

In [ ]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

 pandas.melt can be used without any group identifiers, too:

In [ ]:
pd.melt(df, value_vars=["A", "B", "C"])
pd.melt(df, value_vars=["key", "A", "B"])